In [9]:
import os, re, json, warnings
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA

warnings.filterwarnings("ignore", category=UserWarning)
mne.set_log_level("WARNING")

# ----- Rutas (misma firma que tenías)
REPO = Path("..").resolve()
DATA_RAW  = REPO / "data" / "raw"
DATA_PROC = REPO / "data" / "processed"
FIG_DIR   = DATA_PROC / "qc_figs"
for d in (DATA_PROC, FIG_DIR): d.mkdir(parents=True, exist_ok=True)

print("RAW  →", DATA_RAW)
print("PROC →", DATA_PROC)
print("FIGS →", FIG_DIR)

# ----- Exclusiones (por número)
EXCLUDE_SUBJECTS = {38, 88, 89, 92, 100, 104}

# ----- Objetivo 4 clases (sólo MI)
TARGET_CLASSES = ['left', 'right', 'both_hands', 'both_feet']

# ----- Canales motores (8)
MOTOR_8 = ['C3','CZ','C4','FC3','FC4','CP3','CPZ','CP4']

# Event ID GLOBAL, fijo para TODOS los runs
EVENT_ID_GLOBAL = {
    'left': 1,
    'right': 2,
    'both_hands': 3,
    'both_feet': 4,
}


# ----- Config global
CFG = dict(
    sfreq_target = 160.0,
    tmin_epoch   = 0.0,
    tmax_epoch   = 4.5,       # guardamos 0–4.5; los modelos luego pueden .crop(0.5,3.0) etc.
    bandpass     = (4., 38.), # opcional; pon None para desactivar
    notch        = 60,      # 50/60 si lo necesitas
    notch_method='iir',       # 'fir' (default) o 'iir'
    iir_params=dict(ftype='butter', order=4),  # solo si usas 'iir'
    reref        = None, # o None
    ica_enable   = True,     # ICA opcional (rápida)
    ica_ncomp    = 0.99,
    ica_random   = 97,
    motor_only   = True,      # sólo 8 canales motores
    keep_all_chs = False
)

def _sid_to_int(sid: str) -> int:
    try: return int(sid[1:4])
    except: return -1


RAW  → /root/Proyecto/EEG_Clasificador/data/raw
PROC → /root/Proyecto/EEG_Clasificador/data/processed
FIGS → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs


In [10]:
def list_subjects(raw_dir: Path = DATA_RAW):
    """
    Devuelve ['S001', ...] soportando:
      - raw/physionet_MI/S001/S001R*.edf
      - raw/physionet_MI/S001R*.edf
    Aplica EXCLUDE_SUBJECTS.
    """
    subs = set()

    # Subcarpetas S???
    for p in raw_dir.glob("S???"):
        if p.is_dir() and re.fullmatch(r"S\d{3}", p.name):
            if any(p.glob(f"{p.name}R*.edf")):
                subs.add(p.name)

    # Plano
    for f in raw_dir.glob("S???R*.edf"):
        m = re.match(r"(S\d{3})R\d{2}\.edf$", f.name)
        if m: subs.add(m.group(1))

    subs = sorted(s for s in subs if _sid_to_int(s) not in EXCLUDE_SUBJECTS)
    print(f"[Sujets] encontrados={len(subs)} | excluidos={sorted(EXCLUDE_SUBJECTS)}")
    return subs


def find_edf_runs_for_subject(sid: str, raw_dir: Path = DATA_RAW):
    """
    Lista rutas EDF del sujeto, priorizando subcarpeta raw/.../S001/S001R*.edf.
    """
    subdir = raw_dir / sid
    edfs = sorted(subdir.glob(f"{sid}R*.edf")) if subdir.exists() else []
    if not edfs:
        edfs = sorted(raw_dir.glob(f"{sid}R*.edf"))
    edfs = [p for p in edfs if re.match(rf"{sid}R\d{{2}}\.edf$", p.name)]
    return edfs


def _run_number_from_name(path: Path) -> int:
    # S001R04.edf -> 4
    m = re.match(r"S\d{3}R(\d{2})\.edf$", path.name)
    return int(m.group(1)) if m else -1


In [11]:
# --- Constantes de runs (imagery) ---
LR_RUNS   = {4, 8, 12}       # Left/Right imagery
BHBF_RUNS = {6, 10, 14}      # Both Hands / Both Feet imagery
MI_RUNS   = LR_RUNS | BHBF_RUNS

def is_mi_run(run_id: int) -> bool:
    return run_id in MI_RUNS

def map_event_to_label(run_id: int, desc_norm: str):
    """desc_norm: 'T0'|'T1'|'T2' (mayúsculas). Devuelve 'left'/'right'/'both_hands'/'both_feet' o None."""
    if desc_norm not in ('T1', 'T2'):
        return None  # ignoramos T0 (rest)
    if run_id in LR_RUNS:
        return 'left' if desc_norm == 'T1' else 'right'
    if run_id in BHBF_RUNS:
        return 'both_hands' if desc_norm == 'T1' else 'both_feet'
    return None


In [12]:
# -------- Utils de canales motores (8 principales) --------
import re

def _normalize_ch_name(name: str):
    """Normaliza nombres de canal (remueve ., minúsculas, etc.)"""
    return re.sub(r'[^A-Za-z0-9]', '', name).upper()

def _pick_motor_indices(info: mne.Info, motor_names=MOTOR_8):
    """
    Devuelve índices de canales motores incluso si tienen sufijos como '.' o '..'.
    """
    up = [_normalize_ch_name(c) for c in info.ch_names]
    target = [_normalize_ch_name(t) for t in motor_names]
    picks = [i for i, c in enumerate(up) if c in target]
    return sorted(set(picks))


# -------- Filtros básicos con orden recomendado --------
def apply_basic_filters(raw: mne.io.BaseRaw, cfg=CFG):
    """
    Orden recomendado:
      1) Notch (50/60 Hz y armónicos)
      2) Band-pass (p.ej. 4-40 Hz)
      3) CAR (average reference)    <-- AQUÍ aplicamos el CAR
      4) Resample (si hace falta)
      5) Selección de 8 canales MI (opcional)

    Notas:
      - El notch usa FIR por defecto. Si en el cfg defines:
            cfg['notch_method'] = 'iir'
            cfg['iir_params'] = dict(ftype='butter', order=4)
        entonces el notch será IIR.
      - Deja cfg['reref']=None para no aplicar CAR.
    """
    # 1) Notch
    if cfg.get('notch') is not None:
        notch_method = cfg.get('notch_method', 'fir')  # 'fir' (default) o 'iir'
        if notch_method == 'iir':
            iir_params = cfg.get('iir_params', dict(ftype='butter', order=4))
            raw.notch_filter(cfg['notch'], picks='eeg', method='iir',
                             iir_params=iir_params, verbose=False)
        else:
            raw.notch_filter(cfg['notch'], picks='eeg', method='fir',
                             fir_design='firwin', verbose=False)

    # 2) Band-pass
    if cfg.get('bandpass') is not None:
        fmin, fmax = cfg['bandpass']
        raw.filter(fmin, fmax, picks='eeg',
                   l_trans_bandwidth='auto', h_trans_bandwidth='auto',
                   method='fir', fir_design='firwin', verbose=False)

    # 3) CAR (después del filtrado)
    if cfg.get('reref', None) == 'average':
        raw.set_eeg_reference('average', projection=False, verbose=False)

    # 4) Resample (si difiere)
    if 'sfreq_target' in cfg and abs(raw.info['sfreq'] - cfg['sfreq_target']) > 1e-6:
        raw.resample(cfg['sfreq_target'], npad="auto", verbose=False)

    # 5) Selección de 8 canales motores (si procede)
    if cfg.get('motor_only', True) and not cfg.get('keep_all_chs', False):
        picks = _pick_motor_indices(raw.info, MOTOR_8)
        if picks:
            raw.pick(picks=picks)
    
    rename_map = {ch: re.sub(r'[^A-Za-z0-9]', '', ch).upper() for ch in raw.ch_names}
    raw.rename_channels(rename_map)

    return raw


# -------- ICA opcional (después de fijar referencia) --------
def maybe_run_ica(raw: mne.io.BaseRaw, cfg=CFG):
    """
    ICA opcional. Se recomienda correr tras filtrar y con referencia fija.
    Por simplicidad, entrenamos la ICA con una copia high-pass 1 Hz (establece mejor los componentes),
    y aplicamos la solución al raw original filtrado.
    """
    if not cfg.get('ica_enable', False):
        return raw, None

    ica = ICA(n_components=cfg.get('ica_ncomp', 20),
              random_state=cfg.get('ica_random', 97),
              method='fastica', max_iter=300)

    # Entrena ICA con HP 1 Hz (evita drift). No toques el raw original.
    raw_for_ica = raw.copy().filter(l_freq=1., h_freq=None, picks='eeg', verbose=False)
    ica.fit(raw_for_ica, verbose=False)

    # Aplica ICA (si no marcas comp. a excluir, aplica todo tal cual)
    raw_out = raw.copy()
    ica.apply(raw_out, verbose=False)
    return raw_out, ica


In [13]:
def make_epochs_from_run(path_edf: Path, cfg=CFG):
    run_id = _run_number_from_name(path_edf)  # extrae 1..14 desde '...R04.edf'
    if run_id < 0 or not is_mi_run(run_id):
        return None

    raw = mne.io.read_raw_edf(str(path_edf), preload=True, verbose=False)
    raw = apply_basic_filters(raw, cfg)
    raw, _ = maybe_run_ica(raw, cfg)

    ann = raw.annotations
    if ann is None or len(ann) == 0:
        return None

    # ---------------- mapeo T1/T2 -> etiqueta usando el run_id ----------------
    events = []
    for onset, desc in zip(ann.onset, ann.description):
        lab = map_event_to_label(run_id, str(desc).strip().upper())
        if lab is not None:
            events.append((onset, lab))

    if not events:
        return None

    # ---------------- (clave) SIEMPRE codifica con el EVENT_ID_GLOBAL ---------
    sf = raw.info['sfreq']
    mne_events = np.array(
        [[int(onset_sec * sf), 0, EVENT_ID_GLOBAL[lab]] for onset_sec, lab in events],
        dtype=int
    )

    # ---------------- (clave) Pasa el EVENT_ID_GLOBAL completo ----------------
    epochs = mne.Epochs(
        raw, mne_events, event_id=EVENT_ID_GLOBAL,
        tmin=cfg['tmin_epoch'], tmax=cfg['tmax_epoch'],
        baseline=None, picks='eeg', preload=True, verbose=False,
        on_missing='ignore',  # ignora que en este run falten clases
        event_repeated='drop'
    )
    return epochs





def preprocess_subject(sid: str, cfg=CFG, raw_dir: Path = DATA_RAW, out_dir: Path = DATA_PROC):
    if _sid_to_int(sid) in EXCLUDE_SUBJECTS:
        print(f"[SKIP] {sid} excluido por lista.")
        return None

    edfs = find_edf_runs_for_subject(sid, raw_dir=raw_dir)
    edfs_mi = [p for p in edfs if is_mi_run(_run_number_from_name(p))]
    if not edfs_mi:
        print(f"[WARN] {sid} sin runs MI.")
        return None

    ep_list = []
    for p in edfs_mi:
        ep = make_epochs_from_run(p, cfg=cfg)
        if ep is not None and len(ep):
            ep_list.append(ep)

    if not ep_list:
        print(f"[WARN] {sid} sin epochs construidos.")
        return None

    # Alinear canales por si difieren entre runs
    chs = ep_list[0].ch_names
    for e in ep_list[1:]:
        if e.ch_names != chs:
            e.reorder_channels(chs)
    epochs = mne.concatenate_epochs(ep_list, on_mismatch='ignore')

    out_fif = out_dir / f"{sid}_MI-epo.fif"
    epochs.save(str(out_fif), overwrite=True)
    print(f"[OK] {sid} → {out_fif} | n_epochs={len(epochs)} | ch={len(epochs.ch_names)}")
    return out_fif


In [14]:
def qc_quick(epochs: mne.Epochs, sid: str, fig_dir: Path = FIG_DIR):
    fig_dir.mkdir(parents=True, exist_ok=True)
    # PSD promedio
    try:
        fig = epochs.plot_psd(fmax=60, average=True, show=False)
        out = fig_dir / f"{sid}_PSD.png"
        fig.savefig(out, dpi=130); plt.close(fig)
        print(f"[QC] PSD → {out}")
    except Exception as e:
        print(f"[QC] PSD error {sid}: {e}")

    # Conteo de clases
    inv = {v:k for k,v in epochs.event_id.items()}
    labels = [inv[e[-1]] for e in epochs.events]
    ser = pd.Series(labels).value_counts().reindex(TARGET_CLASSES, fill_value=0)
    ser.to_csv(fig_dir / f"{sid}_counts.csv")


In [15]:
def debug_list_runs_and_events(subject_dir: Path):
    edfs = sorted(subject_dir.glob("S???R??.edf"))
    print(f"[DEBUG] {subject_dir.name}: {len(edfs)} EDFs")
    for p in edfs:
        run_id = _run_number_from_name(p)
        tag = "LR" if run_id in LR_RUNS else "BH/BF" if run_id in BHBF_RUNS else "OTHER"
        try:
            raw = mne.io.read_raw_edf(str(p), preload=False, verbose=False)
            ann = raw.annotations
            if ann is None: 
                print(f"  - {p.name} (R{run_id:02d}) [{tag}] -> sin anotaciones")
                continue
            labs = []
            for desc in ann.description:
                lab = map_event_to_label(run_id, str(desc).strip().upper())
                if lab is not None:
                    labs.append(lab)
            from collections import Counter
            c = Counter(labs)
            if is_mi_run(run_id):
                print(f"  - {p.name} (R{run_id:02d}) [{tag}] -> {dict(c)}")
        except Exception as e:
            print(f"  - {p.name} ERROR: {e}")

# Ejemplo de uso para un sujeto:
debug_list_runs_and_events(DATA_RAW / "S001")


[DEBUG] S001: 14 EDFs
  - S001R04.edf (R04) [LR] -> {'right': 7, 'left': 8}
  - S001R06.edf (R06) [BH/BF] -> {'both_feet': 8, 'both_hands': 7}
  - S001R08.edf (R08) [LR] -> {'left': 8, 'right': 7}
  - S001R10.edf (R10) [BH/BF] -> {'both_hands': 7, 'both_feet': 8}
  - S001R12.edf (R12) [LR] -> {'right': 8, 'left': 7}
  - S001R14.edf (R14) [BH/BF] -> {'both_feet': 8, 'both_hands': 7}


In [16]:
def preprocess_all(cfg=CFG, raw_dir: Path = DATA_RAW, out_dir: Path = DATA_PROC, do_qc=True):
    subs = list_subjects(raw_dir)
    rows = []
    for sid in subs:
        out = preprocess_subject(sid, cfg=cfg, raw_dir=raw_dir, out_dir=out_dir)
        if out is None: 
            continue
        ep = mne.read_epochs(str(out), preload=False, verbose=False)
        inv = {v:k for v,k in enumerate([])}  # placeholder no usado

        # QC (ligero)
        if do_qc:
            ep.load_data()
            qc_quick(ep, sid, fig_dir=FIG_DIR)

        inv = {v:k for k,v in ep.event_id.items()}
        labels = [inv[e[-1]] for e in ep.events]
        cnt = pd.Series(labels).value_counts().reindex(TARGET_CLASSES, fill_value=0)

        rows.append(dict(subject=sid, n_epochs=len(ep), n_ch=len(ep.ch_names),
                         left=int(cnt['left']), right=int(cnt['right']),
                         both_hands=int(cnt['both_hands']), both_feet=int(cnt['both_feet'])))

    df = pd.DataFrame(rows).sort_values('subject')
    out_csv = out_dir / "preproc_summary.csv"
    df.to_csv(out_csv, index=False)
    print(f"[RESUMEN] sujetos OK={len(df)}/{len(subs)} | CSV → {out_csv}")
    try:
        display(df.head(10))
    except:
        pass
    return df

# Ejecuta cuando quieras:
df_summary = preprocess_all(CFG, DATA_RAW, DATA_PROC, do_qc=True)


[Sujets] encontrados=103 | excluidos=[38, 88, 89, 92, 100, 104]


<ipython-input-13-83caa3bf9fe7>:71: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(ep_list, on_mismatch='ignore')


[OK] S001 → /root/Proyecto/EEG_Clasificador/data/processed/S001_MI-epo.fif | n_epochs=90 | ch=8
[QC] PSD → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs/S001_PSD.png


<ipython-input-13-83caa3bf9fe7>:71: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(ep_list, on_mismatch='ignore')


[OK] S002 → /root/Proyecto/EEG_Clasificador/data/processed/S002_MI-epo.fif | n_epochs=84 | ch=8
[QC] PSD → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs/S002_PSD.png


...

[OK] S107 → /root/Proyecto/EEG_Clasificador/data/processed/S107_MI-epo.fif | n_epochs=90 | ch=8
[QC] PSD → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs/S107_PSD.png


<ipython-input-13-83caa3bf9fe7>:71: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(ep_list, on_mismatch='ignore')


[OK] S108 → /root/Proyecto/EEG_Clasificador/data/processed/S108_MI-epo.fif | n_epochs=84 | ch=8
[QC] PSD → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs/S108_PSD.png
[OK] S109 → /root/Proyecto/EEG_Clasificador/data/processed/S109_MI-epo.fif | n_epochs=84 | ch=8
[QC] PSD → /root/Proyecto/EEG_Clasificador/data/processed/qc_figs/S109_PSD.png
[RESUMEN] sujetos OK=103/103 | CSV → /root/Proyecto/EEG_Clasificador/data/processed/preproc_summary.csv


<ipython-input-13-83caa3bf9fe7>:71: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(ep_list, on_mismatch='ignore')


,subject,n_epochs,n_ch,left,right,both_hands,both_feet
0,S001,90,8,23,22,21,24
1,S002,84,8,21,21,21,21
2,S003,90,8,23,22,21,24
3,S004,84,8,21,21,21,21
4,S005,84,8,21,21,21,21
5,S006,84,8,21,21,21,21
6,S007,90,8,23,22,22,23
7,S008,84,8,21,21,21,21
8,S009,84,8,21,21,21,21
9,S010,84,8,21,21,21,21


In [17]:
def quick_count_subject_epochs(fif_path: Path):
    ep = mne.read_epochs(str(fif_path), preload=False, verbose=False)
    inv = {v:k for k,v in ep.event_id.items()}
    y = [inv[e[-1]] for e in ep.events]
    from collections import Counter
    return Counter(y)

# Ejemplo de uso:
for fif in sorted(DATA_PROC.glob("S???_MI-epo.fif")):
    print(fif.name, dict(quick_count_subject_epochs(fif)))


S001_MI-epo.fif {'right': 22, 'left': 23, 'both_feet': 24, 'both_hands': 21}
S002_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S003_MI-epo.fif {'right': 22, 'left': 23, 'both_feet': 24, 'both_hands': 21}
S004_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S005_MI-epo.fif {'right': 21, 'left': 21, 'both_hands': 21, 'both_feet': 21}
S006_MI-epo.fif {'left': 21, 'right': 21, 'both_feet': 21, 'both_hands': 21}
S007_MI-epo.fif {'left': 23, 'right': 22, 'both_hands': 22, 'both_feet': 23}
S008_MI-epo.fif {'left': 21, 'right': 21, 'both_feet': 21, 'both_hands': 21}
S009_MI-epo.fif {'right': 21, 'left': 21, 'both_feet': 21, 'both_hands': 21}
S010_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S011_MI-epo.fif {'left': 21, 'right': 21, 'both_feet': 21, 'both_hands': 21}


S012_MI-epo.fif {'left': 21, 'right': 21, 'both_feet': 21, 'both_hands': 21}
S013_MI-epo.fif {'right': 21, 'left': 21, 'both_feet': 21, 'both_hands': 21}
S014_MI-epo.fif {'left': 21, 'right': 21, 'both_feet': 21, 'both_hands': 21}
S015_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S016_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S017_MI-epo.fif {'right': 21, 'left': 21, 'both_hands': 21, 'both_feet': 21}
S018_MI-epo.fif {'right': 21, 'left': 21, 'both_feet': 21, 'both_hands': 21}
S019_MI-epo.fif {'right': 21, 'left': 21, 'both_hands': 21, 'both_feet': 21}
S020_MI-epo.fif {'left': 21, 'right': 21, 'both_hands': 21, 'both_feet': 21}
S021_MI-epo.fif {'right': 21, 'left': 24, 'both_feet': 21, 'both_hands': 24}
S022_MI-epo.fif {'left': 22, 'right': 23, 'both_hands': 21, 'both_feet': 24}
S023_MI-epo.fif {'right': 21, 'left': 21, 'both_hands': 21, 'both_feet': 21}
S024_MI-epo.fif {'right': 21, 'left': 21, 'both_hands': 21, 'both_feet': 21}

In [18]:
from pathlib import Path
import mne, pandas as pd

# Ajusta esta ruta a tu carpeta processed
DATA_PROC = Path("../data/processed")

# Los 8 canales esperados (ajusta si tu lista MOTOR_8 es distinta)
EXPECTED_8 = ['C3', 'CZ', 'C4', 'FC3', 'FC4', 'CP3', 'CPZ', 'CP4']
EXPECTED_8_UP = [c.upper() for c in EXPECTED_8]

def check_one(fif_path: Path):
    ep = mne.read_epochs(str(fif_path), preload=False, verbose=False)
    chs = ep.info["ch_names"]
    ok_len = (len(chs) == 8)
    ok_set = ([c.upper() for c in chs] == EXPECTED_8_UP) or (set([c.upper() for c in chs]) == set(EXPECTED_8_UP))
    return chs, ok_len, ok_set

# 1) Muestra rápida en un archivo (por ejemplo S001)
one = next(DATA_PROC.glob("S???_MI-epo.fif"), None)
if one:
    chs, ok_len, ok_set = check_one(one)
    print(f"[Quick check] {one.name} → n_ch={len(chs)} | ok_len={ok_len} | ok_set={ok_set}")
    print("Channels:", chs)
else:
    print("No se encontraron FIFs en", DATA_PROC)

# 2) Revisión masiva y reporte CSV
rows = []
for fif in sorted(DATA_PROC.glob("S???_MI-epo.fif")):
    chs, ok_len, ok_set = check_one(fif)
    rows.append(dict(
        subject=fif.stem.split("_")[0],
        file=fif.name,
        n_channels=len(chs),
        exact_8=ok_len,
        match_expected=ok_set,
        channels=",".join(chs)
    ))

df = pd.DataFrame(rows).sort_values("subject")
out_csv = DATA_PROC / "qc_channels_report.csv"
df.to_csv(out_csv, index=False)
print("QC canales →", out_csv)

# 3) Resumen rápido
print(df.groupby(["exact_8","match_expected"]).size().rename("count"))




[Quick check] S065_MI-epo.fif → n_ch=8 | ok_len=True | ok_set=True
Channels: ['FC3', 'FC4', 'C3', 'CZ', 'C4', 'CP3', 'CPZ', 'CP4']
QC canales → ../data/processed/qc_channels_report.csv
exact_8  match_expected
True     True              103
Name: count, dtype: int64
